<a href="https://colab.research.google.com/github/AlvaroMartinez87/game-of-life/blob/master/CUDA_en_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# USO DE CUDA EN GOOGLE COLAB

## Configuración del entorno de ejecución 

Habilita la GPU en el entorno de ejecución de Colab:
1. En el menú: `Editar/Configuración del cuaderno`
2. Selecciona **GPU** en el menú desplegable `Acelerador por hardware`

Muestra información sobre la arquitectura de las GPUs disponibles en la máquina.

In [ ]:
!nvidia-smi

Wed May 18 17:10:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------


Muestra la versión de CUDA

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


Clona el repositorio de ejemplos de CUDA runtime

In [ ]:
!git clone https://github.com/NVIDIA/cuda-samples

Cloning into 'cuda-samples'...
remote: Enumerating objects: 9693, done.
remote: Total 9693 (delta 0), reused 0 (delta 0), pack-reused 9693
Receiving objects: 100% (9693/9693), 122.53 MiB | 29.95 MiB/s, done.
Resolving deltas: 100% (7840/7840), done.
Checking out files: 100% (3612/3612), done.


Haz que `cuda-samples` sea tu directorio de trabajo

In [ ]:
cd cuda-samples

/content/cuda-samples


Muestra lista de tags del repositorio de `cuda-samples`

In [ ]:
!git tag

10.1.1
10.1.2
v10.0
v10.0.1
v10.1
v10.2
v11.0
v11.1
v11.2
v11.3
v11.4
v11.4.1
v11.5
v11.6
v9.2


Cambia a la rama del número de versión que coincida con la versión de CUDA runtime mostrada anteriormente (en el caso de creación de este cuaderno fue la v11.1                 

In [ ]:
!git checkout tags/v11.1

Note: checking out 'tags/v11.1'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at c4e2869 add multi-warp cooperative groups based reduction kernel in reduction sample


Haz que `Samples/deviceQuery` sea tu directorio de trabajo

In [ ]:
cd Samples/deviceQuery

/content/cuda-samples/Samples/deviceQuery


Compila y ejecuta el programa deviceQuery. Observa el nombre de la GPU, las versiones del driver de CUDA y de CUDA runtime, la capacidad de cómputo (CUDA Capability), la cantidad de memoria, el número de cores CUDA, dimensiones máximas de bloque y grid…

In [ ]:
!make

/usr/local/cuda/bin/nvcc -ccbin g++ -I../../Common  -m64    -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_86,code=compute_86 -o deviceQuery.o -c deviceQuery.cpp
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
/usr/local/cuda/bin/nvcc -ccbin g++   -m64      -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,

Ejecuta el programa `deviceQuery`. Observa el nombre de la GPU, las versiones del driver de CUDA y de CUDA runtime, la capacidad de cómputo (CUDA Capability), la cantidad de memoria, el número de cores CUDA, dimensiones máximas de bloque y grid…

In [ ]:
!./deviceQuery

./deviceQuery Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla T4"
  CUDA Driver Version / Runtime Version          11.2 / 11.1
  CUDA Capability Major/Minor version number:    7.5
  Total amount of global memory:                 15110 MBytes (15843721216 bytes)
  (40) Multiprocessors, ( 64) CUDA Cores/MP:     2560 CUDA Cores
  GPU Max Clock rate:                            1590 MHz (1.59 GHz)
  Memory Clock rate:                             5001 Mhz
  Memory Bus Width:                              256-bit
  L2 Cache Size:                                 4194304 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=(131072), 2D=(131072, 65536), 3D=(16384, 16384, 16384)
  Maximum Layered 1D Texture Size, (num) layers  1D=(32768), 2048 layers
  Maximum Layered 2D Texture Size, (num) layers  2D=(32768, 32768), 2048 layers
  Total amount of constant memory:               65536 bytes
  Total amount of shar

## EJEMPLO: SAXPY

Haz que `content` sea tu directorio de trabajo

In [ ]:
cd /content

/content


Descarga el archivo `saxpyCPU.cpp`

In [ ]:
!wget -O saxpyCPU.cpp https://www.dropbox.com/s/fzaph95uz4qlvju/saxpyCPU.cpp?dl=0

--2022-05-18 17:20:48--  https://www.dropbox.com/s/fzaph95uz4qlvju/saxpyCPU.cpp?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fzaph95uz4qlvju/saxpyCPU.cpp [following]
--2022-05-18 17:20:48--  https://www.dropbox.com/s/raw/fzaph95uz4qlvju/saxpyCPU.cpp
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca64593e0b6c212a209690d8799.dl.dropboxusercontent.com/cd/0/inline/BljTgJOWknyKseSxNnTBZTwB7rmyPObYOnJt9IoM-hzllKMyjELUDi0qZfJ8L4jTm1ReH7Q7HXqA-_bQt2xecEDa98YYMoOeGqzbBPqXq2QwRcpCzsKlbmvcGl5Ndq8_9u9b8ohBBr0Km5c1EBDobYpUXAj5dIFa2WCXMQGexjpT9Q/file# [following]
--2022-05-18 17:20:48--  https://uca64593e0b6c212a209690d8799.dl.dropboxusercontent.com/cd/0/inline/BljTgJOWknyKseSxNnTBZTwB7rmyPObYOnJt9IoM-hzllKMyjELUDi0qZfJ8L4j

Lee el contenido del archivo `saxpyCPU.cpp`

In [ ]:
!cat saxpyCPU.cpp

#include <stdio.h>
#include <stdlib.h>

// Función que calcula Y=a*X+Y
void saxpy_cpu(float *x, float *y, float a, int numElementos)
{
    for (int i = 0; i < numElementos; ++i)
    {
        y[i] = a*x[i] + y[i];
    }
}

int main(void)
{
    // Definimos tamaños: 1M de elementos (2^20)
    int numElementos = 1<<20; // 
    size_t tamano = numElementos * sizeof(float);
	
    //Definimos variables
    float *x, *y;
	
	// Reservamos memoria
    x = (float *)malloc(tamano);
    y = (float *)malloc(tamano);
    
    // Asignamos valores
    for (int i = 0; i < numElementos; i++)
    {
        x[i] = 1.0f;
        y[i] = 2.0f;
    }
	float a = 5.0f;

    // LLamamos a función saxpy_cpu
    saxpy_cpu(x, y, a, numElementos);
    
    // Verificamos resultado
    float Error = 0.0f;
    for (int i = 0; i < numElementos; i++)
        Error = Error + abs(y[i]-7.0f);
    printf("Error: %f\n", Error);

    // Liberamos memoria host
    free(x);
    free(y);

    return 0;
}



Compila el archivo `saxpyCPU.cpp`

In [ ]:
!g++ saxpyCPU.cpp -o saxpyCPU

Ejecuta el programa `saxpyCPU`

In [ ]:
!./saxpyCPU

Error: 0.000000


Descarga el archivo `saxpy.cu`

In [ ]:
!wget -O saxpy.cu https://www.dropbox.com/s/nmm0u2cz5ayclpw/saxpy.cu?dl=0

--2022-05-18 17:27:15--  https://www.dropbox.com/s/nmm0u2cz5ayclpw/saxpy.cu?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/nmm0u2cz5ayclpw/saxpy.cu [following]
--2022-05-18 17:27:15--  https://www.dropbox.com/s/raw/nmm0u2cz5ayclpw/saxpy.cu
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9b89e027e85d25a8acba2c11ed.dl.dropboxusercontent.com/cd/0/inline/BlhOIX_MQZKkbhuFGpiBx4FrprJunNWOS932h_5DV6gF5o19j8FOmxEg2-23IXI-dlB3bdnqQP2AoRwqkxnrHmFbpaUqAya-w9Jp4Z1pzf7XsfzilxHpFc506T-8xL7aW1qFux03TosB14qmXCVtTwWdYdKuZ5apcCPTV5GWMd-uSA/file# [following]
--2022-05-18 17:27:16--  https://uc9b89e027e85d25a8acba2c11ed.dl.dropboxusercontent.com/cd/0/inline/BlhOIX_MQZKkbhuFGpiBx4FrprJunNWOS932h_5DV6gF5o19j8FOmxEg2-23IXI-dlB3bdnqQP2

Edita y completa el programa saxpy.cu
(doble clic sobre el archivo en el panel de la izquierda)

Compila el programa `saxpy.cu`

In [ ]:
!nvcc -arch=sm_75 saxpy.cu -o saxpy

Ejecuta el programa saxpy

In [ ]:
!./saxpy

Error: 0.000000
